# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
keras = tf.keras
from sklearn.preprocessing import RobustScaler
import plotly.graph_objects as go
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Importing Data

In [3]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

# Xử lý ngày

In [4]:
confirmed_df = confirmed_df.groupby("Country/Region").sum().reset_index()
Country_confirmed =  confirmed_df['Country/Region'].values
confirmed_df = confirmed_df.drop(['Lat','Long','Country/Region'], axis=1)
confirmed_df = confirmed_df[confirmed_df.columns].T
confirmed_df.columns = Country_confirmed
Date_index = confirmed_df.index
confirmed_df = confirmed_df.diff().fillna(0)
confirmed_df = confirmed_df.abs()
confirmed_df.head()

C:\Users\THAIHB.B19CN638\AppData\Local\Temp\ipykernel_2924\3785689273.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  confirmed_df = confirmed_df.groupby("Country/Region").sum().reset_index()


,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
1/22/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/23/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
1/24/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/25/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/26/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Thu nhỏ dữ liệu

In [5]:
sc = RobustScaler()
confirmed_df_scaled = sc.fit_transform(confirmed_df)
confirmed_df_scaled = pd.DataFrame(confirmed_df_scaled)

# Nhập số ngày bạn muốn dự đoán

In [6]:
DAYS_TO_PREDICT = int(input('Enter Number Of Days You Want to Predict : '))
predict_confirm_index = pd.date_range(start=Date_index[-1],periods=DAYS_TO_PREDICT + 1,closed='right')
predict_confirm_index = predict_confirm_index.strftime('%Y/%m/%d')

Enter Number Of Days You Want to Predict : 100


C:\Users\THAIHB.B19CN638\AppData\Local\Temp\ipykernel_2924\1395712551.py:2: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  predict_confirm_index = pd.date_range(start=Date_index[-1],periods=DAYS_TO_PREDICT + 1,closed='right')


# Tạo lớp xây dựng mô hình

In [7]:
class Corona_Confirmed_model():
    def create_dataset(self, dataset, time_steps=1, column = 0):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_steps-1):
          a = dataset[i:(i+time_steps), column]
          dataX.append(a)
          dataY.append(dataset[i + time_steps, column])
        return np.array(dataX), np.array(dataY)

    def Model(self,dframe,column):
        df = dframe.values
        df = df.astype('float32')
        
        time_steps = 1
        trainX, trainY = self.create_dataset(df, time_steps, column)
        trainX = np.reshape(trainX, (trainX.shape[0],1,trainX.shape[1]))
        
        
        keras.backend.clear_session()
        tf.random.set_seed(42)
        np.random.seed(42)
        confirm_model = keras.models.Sequential([
            keras.layers.InputLayer(input_shape=[None, 1]),
            keras.layers.Conv1D(filters=16, kernel_size=2,
                      strides=1, padding="causal",
                      activation="relu",
                      ),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(1,activation = 'linear',kernel_initializer = 'he_normal'),
            keras.layers.Lambda(lambda x: x * 200)
             ])
        
        optimizer = keras.optimizers.Adadelta(lr=0.001)
        
        confirm_model.compile(loss=['mae'],optimizer=optimizer)   
                
        confirm_model.fit(trainX, trainY, epochs=100,
                          batch_size = 10,verbose = 0)
        

        confirm_test_seq  = trainX[-1:]
        confirm_test_seq = confirm_test_seq
        confirm_test_seq = confirm_test_seq.reshape(len(confirm_test_seq), time_steps, 1)
        confirm_preds = []
        
        for _ in range (DAYS_TO_PREDICT):
            confirm_pred = confirm_model.predict(confirm_test_seq)
            confirm_preds.append(confirm_pred)
            confirm_new_seq = confirm_pred
            confirm_test_seq = confirm_new_seq
            confirm_test_seq = confirm_test_seq.reshape(len(confirm_test_seq ), time_steps, 1)

        

        return confirm_preds

# Gọi lớp và dự đoán các trường hợp cho từng quốc gia

In [8]:
# NN = Corona_Confirmed_model()
# confirm_predictions = []
# for i in range(len(Country_confirmed)):
#     result = NN.Model(confirmed_df_scaled,i) 
#     confirm_predictions.append(result)

1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 34ms/step


...

1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 27ms/step


In [3]:
confirm_predictions

NameError: name 'confirm_predictions' is not defined

# Xử lý dữ liệu dự đoán

In [11]:
# confirm_predictions = (np.array(confirm_predictions).transpose())
# confirm_predictions = confirm_predictions.reshape(((DAYS_TO_PREDICT),len(Country_confirmed)))
# confirm_predictions = sc.inverse_transform(confirm_predictions)
# confirm_predictions = pd.DataFrame(confirm_predictions)
# confirm_predictions = confirm_predictions.abs()
# confirm_predictions['Dates'] = predict_confirm_index
# confirm_predictions.set_index('Dates', inplace = True)
# confirm_predictions.columns = Country_confirmed
# confirm_predictions.head()

,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
Dates,,,,,,,,,,,,,,,,,,,,,
2022/11/27,15225.375977,38534.082031,25466.027344,2898.125000,8745.626953,76.791695,168.664490,8.382268e+05,41219.769531,7.761060e+05,...,9.974109e+04,4.371185e+04,131.362015,1.131890e+05,6.475848e+05,9.344653e+04,131.362015,1578.344238,34607.710938,20516.390625
2022/11/28,26267.539062,74915.523438,51781.269531,6024.989746,18773.431641,172.037811,389.944397,1.933179e+06,95266.460938,1.797321e+06,...,2.365369e+02,1.489943e+02,0.170036,3.825548e+02,1.230131e+03,2.311651e+02,0.128876,2.405746,5887.109375,4802.963867
2022/11/29,5564.392090,14901.138672,9778.039062,1105.860107,3375.356689,29.884592,67.238785,3.355044e+05,16520.109375,3.089632e+05,...,1.039392e+02,1.329108e+02,0.001238,3.500636e+02,3.046011e+02,1.803791e+02,0.001238,2.017087,59.867935,36.215164
2022/11/30,78.249451,124.180847,159.450623,0.051780,28.656738,0.001399,0.003149,3.656541e+03,181.765152,2.749562e+02,...,3.748824e+06,1.637257e+06,4942.033203,4.239560e+06,2.435102e+07,3.504228e+06,174.326691,2305.872803,63602.855469,42091.484375
2022/12/01,53495.136719,141136.562500,94805.914062,10814.900391,32840.398438,292.692200,658.805176,3.254715e+06,160256.265625,3.024558e+06,...,6.762858e+06,6.650336e+06,8915.981445,1.721971e+07,4.393369e+07,1.391857e+06,1965.685059,23579.144531,516832.125000,302137.750000


In [18]:
confirm_predictions

,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
Dates,,,,,,,,,,,,,,,,,,,,,
2022/11/27,1.522538e+04,3.853408e+04,2.546603e+04,2.898125e+03,8.745627e+03,76.791695,168.664490,8.382268e+05,41219.769531,7.761060e+05,...,9.974109e+04,4.371185e+04,131.362015,1.131890e+05,6.475848e+05,9.344653e+04,131.362015,1578.344238,34607.710938,20516.390625
2022/11/28,2.626754e+04,7.491552e+04,5.178127e+04,6.024990e+03,1.877343e+04,172.037811,389.944397,1.933179e+06,95266.460938,1.797321e+06,...,2.365369e+02,1.489943e+02,0.170036,3.825548e+02,1.230131e+03,2.311651e+02,0.128876,2.405746,5887.109375,4802.963867
2022/11/29,5.564392e+03,1.490114e+04,9.778039e+03,1.105860e+03,3.375357e+03,29.884592,67.238785,3.355044e+05,16520.109375,3.089632e+05,...,1.039392e+02,1.329108e+02,0.001238,3.500636e+02,3.046011e+02,1.803791e+02,0.001238,2.017087,59.867935,36.215164
2022/11/30,7.824945e+01,1.241808e+02,1.594506e+02,5.178003e-02,2.865674e+01,0.001399,0.003149,3.656541e+03,181.765152,2.749562e+02,...,3.748824e+06,1.637257e+06,4942.033203,4.239560e+06,2.435102e+07,3.504228e+06,174.326691,2305.872803,63602.855469,42091.484375
2022/12/01,5.349514e+04,1.411366e+05,9.480591e+04,1.081490e+04,3.284040e+04,292.692200,658.805176,3.254715e+06,160256.265625,3.024558e+06,...,6.762858e+06,6.650336e+06,8915.981445,1.721971e+07,4.393369e+07,1.391857e+06,1965.685059,23579.144531,516832.125000,302137.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023/03/02,1.888794e+08,5.155108e+08,3.412015e+08,3.920636e+07,1.186788e+08,55.414627,165.368683,1.064433e+06,48569.933594,1.062550e+06,...,9.552255e+04,6.399578e+04,75.141541,7.864518e+04,5.437089e+05,1.011378e+05,186.043243,965.110413,24419.822266,17372.742188
2023/03/03,2.677199e+04,7.008535e+04,5.748100e+04,3.183834e+03,2.858526e+01,0.000732,0.001582,3.648483e+03,181.352448,2.668570e+02,...,3.448603e+05,1.126752e+05,454.525085,2.917557e+05,2.239971e+06,2.410396e+05,454.524994,4072.870605,119599.640625,52194.101562
2023/03/04,8.109708e+04,1.651634e+05,1.465161e+05,1.361712e+04,1.414530e+04,143.822250,330.250427,1.639014e+06,80741.468750,1.522461e+06,...,2.846255e+05,5.296259e+04,182.230057,7.563742e+04,1.881770e+06,1.207651e+05,132.674622,4254.110352,33268.082031,61554.015625


# Trực quan hóa các trường hợp được xác nhận cho một quốc gia theo lựa chọn của người dùng

### Danh sách các quốc gia bị nhiễm bệnh

In [19]:
Infected_Countries = pd.DataFrame(Country_confirmed)
Infected_Countries.columns = ['Countries']
# pd.set_option("max_rows", None)
Infected_Countries

,Countries
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
...,...
196,West Bank and Gaza
197,Winter Olympics 2022
198,Yemen
199,Zambia


### Chọn Quốc gia

In [1]:
Country =input('Nhập tên của quốc gia từ danh sách trên mà bạn muốn xem các trường hợp hàng ngày cho đến bây giờ : ')

Nhập tên của quốc gia từ danh sách trên mà bạn muốn xem các trường hợp hàng ngày cho đến bây giờ : Japan


## Cho đến Date Confirm Cases trên cơ sở hàng ngày của coronavirus

In [2]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Các trường hợp hàng ngày được xác nhận','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},
                    xaxis_title = "Ngày",
                    yaxis_title="Số ca nhiễm",
                     )
fig.add_trace(go.Scatter(x=confirmed_df.index, 
                         y=confirmed_df[Country],
                         mode='lines+markers',
                         line=dict(color='blue', width=2)))

NameError: name 'go' is not defined

## Dự đoán các trường hợp được xác nhận nhiễm coronavirus

In [26]:

fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Dự đoán trường hợp được xác nhận','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},
    xaxis_title = "Ngày",
    yaxis_title="Số ca nhiễm")
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=confirm_predictions.index, 
                         y=confirm_predictions[Country],
                         mode='lines+markers',
                         line=dict(color='yellow', width=2)))